# Notebook 03 — Análise Geográfica Completa
Este notebook realiza uma análise geoespacial detalhada dos bairros do município de Vila Velha utilizando dados do TSE e um shapefile de bairros.

## 1. Importação de bibliotecas

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (12, 8)

## 2. Carregamento das bases de dados

In [ ]:
repo = "analise-perfil-eleitor-vila-velha"

# Leitura dos dados eleitorais
df = pd.read_csv(f"{repo}/dados/base_vila_velha.csv")

# Leitura do shapefile
mapa = gpd.read_file(f"{repo}/dados/geodata/bairros_vila_velha.shp")

df.head(), mapa.head()

## 3. Preparação dos dados

In [ ]:
df['BAIRRO'] = df['BAIRRO'].str.upper().str.strip()
mapa['NOME'] = mapa['NOME'].str.upper().str.strip()

df_agr = df.groupby('BAIRRO').agg({
    'QTDE_ELEITORES': 'sum'
}).reset_index()

df_agr.head()

## 4. Junção dos dados geográficos com a base eleitoral

In [ ]:
mapa_merged = mapa.merge(df_agr, left_on='NOME', right_on='BAIRRO', how='left')

mapa_merged['QTDE_ELEITORES'] = mapa_merged['QTDE_ELEITORES'].fillna(0)
mapa_merged.head()

## 5. Mapa Choropleth — Quantidade de Eleitores por Bairro

In [ ]:
fig, ax = plt.subplots()
mapa_merged.plot(column='QTDE_ELEITORES', ax=ax, legend=True)
ax.set_title("Distribuição de Eleitores por Bairro — Vila Velha")
ax.axis('off')

## 6. Top 10 Bairros com mais Eleitores

In [ ]:
top10 = df_agr.sort_values(by='QTDE_ELEITORES', ascending=False).head(10)

sns.barplot(data=top10, y='BAIRRO', x='QTDE_ELEITORES')
plt.title("Top 10 Bairros com mais Eleitores")
plt.xlabel("Eleitores")
plt.ylabel("Bairro")
plt.show()

## 7. Mapa destacando os 10 maiores bairros

In [ ]:
bairros_top10 = top10['BAIRRO'].tolist()
mapa_merged['TOP10'] = mapa_merged['BAIRRO'].isin(bairros_top10)

fig, ax = plt.subplots()
mapa_merged.plot(ax=ax, color="lightgray")
mapa_merged[mapa_merged['TOP10']].plot(ax=ax, color="red")
ax.set_title("Destaque: Top 10 Bairros com mais Eleitores")
ax.axis('off')

## 8. Análise da distribuição espacial


- Bairros com maior densidade eleitoral aparecem destacados no mapa.
- A concentração tende a ocorrer em regiões mais urbanizadas e populosas.
- Mapas temáticos permitem visualizar padrões territoriais que não aparecem apenas em tabelas.

Essa análise pode ser expandida com:
- Comparação entre regiões (Norte, Centro, Praia, Continental)
- Mapa interativo com **Folium**
- Análise de densidade eleitoral (Kernel Density Estimation)


## 9. Mapa Interativo (Folium)

In [ ]:
import folium
from folium.features import GeoJsonTooltip

m = folium.Map(location=[-20.347, -40.287], zoom_start=12)

geojson = folium.GeoJson(
    mapa_merged,
    name="Eleitores por Bairro",
    tooltip=GeoJsonTooltip(
        fields=['NOME', 'QTDE_ELEITORES'],
        aliases=['Bairro', 'Eleitores'],
        localize=True
    ),
    style_function=lambda x: {
        'fillColor': '#3186cc',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5
    }
)

geojson.add_to(m)
m